<a href="https://colab.research.google.com/github/ZhangEliot/EZYJAWritter/blob/master/AIWritter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<font color='blue'>EZYJAWritter -- 一个自动写渔家傲词的LSTM</font>**</br>
<font color='orange'>按顺序运行就好了</br>
启发语句可能会作为开头</br>
也可能不会~</font>

In [23]:
#@title <font color='hotpink'>运行这个可以看看分配到的是啥GPU，也可以不运行</font> { vertical-output: true }
!nvidia-smi

Thu Jul  9 11:56:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |   2657MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title <font color=red><--- 点这儿运行</font> { vertical-output: true }
#@markdown <font color=orange><b>先运行这个！下载预先训练好的模型！</br>不然运行不起来！（废话。。。</b></font>
!git clone https://github.com/ZhangEliot/EZYJAWritter.git
!7z x EZYJAWritter/yja-166.7z.001 -r

In [29]:
#@title <font color=red><--- 点这儿运行</font> { vertical-output: true }
#@markdown <font color=hotpink><b>提供不多于7个汉字的启发语句，然后即可运行！</b></font>
import logging, os
logging.disable(logging.WARNING)
import sys
import numpy as np
from tensorflow.keras.models import load_model

启发语句 = '雪花飘飘' #@param {type: "string"} 
#@markdown ---
MODEL_PATH = 'yja-166'
CHARSET_PATH = 'EZYJAWritter/GB2312.txt'
MAXLEN = 28


def sample(preds, temperature=1.0, size=1):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(size, preds, 1)
    return np.argmax(probas)


def ChangePouncture(s, p='，'):
    return s[:-1] + p


def PrettyLook(words):
    words[0] = ChangePouncture(words[0])
    words[1] = ChangePouncture(words[1])
    words[3] = ChangePouncture(words[3])
    words[4] = ChangePouncture(words[4], '。\n')
    words[5] = ChangePouncture(words[5])
    words[6] = ChangePouncture(words[6])
    words[8] = ChangePouncture(words[8])
    return ''.join(words)


def WritePoem(sentence):
    # Init
    ezWritter = load_model(MODEL_PATH)
    words = []
    rythmic = [8, 8, 8, 4, 8, 8, 8, 8, 4, 8]
    diversity = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
    # Load charset
    with open(CHARSET_PATH, encoding='UTF-8') as f:
        text = f.read()
    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    ###
    try:
        word = sentence
        temperature = diversity[np.random.randint(0, 6)]
        print('正在构思......')
        for i in range(1000):
            x_pred = np.zeros((1, len(sentence), len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1

            preds = ezWritter.predict(x_pred, verbose=0)[0]
            next_index = sample(preds,
                                temperature,
                                np.random.randint(10, 21))
            next_char = indices_char[next_index]
            word += next_char
            if (next_char == '。'):
                if (len(word) == rythmic[len(words)]):
                    words.append(word)
                else:
                    word = ''
            if (next_char == '\n'):
                word = ''
            if (len(words) == 10):
                return PrettyLook(words)
            sentence = sentence[1:] + \
                next_char if len(sentence) >= MAXLEN else sentence + next_char
    except Exception:
        return '请换句启发语再试！'
    return '请换句启发语再试！'


if __name__ == "__main__":
    print(WritePoem(启发语句[:28]))


正在构思......


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


雪花飘飘归一际，天今不是长门赋，日半嫩来花枕好。君林许，玉朝一念声千侣。
绿索清旗双彩水，千舟须地归年蕊，羌管悠桥行满浪。心窈窕，青声衰到相须傲。


In [ ]:
#@title Lv.??? 训练模型用的 { vertical-output: true }
#@markdown <b><font color='red'>仅供EliotZhang使用！</font></br>(bushi，只是需要配置GoogleDrive工作环境</b></br>
#@markdown <b>懒得写介绍了，看代码自己弄吧~</br>也就几个文件夹的事儿</b></br>
#@markdown <b>实在不行可以看看我的工作空间是啥样的：</br>[传送门](https://drive.google.com/drive/folders/1-x8qzErCO_E8TMuGtxiPkhUuQl5iBj1d?usp=sharing)</b>

from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import os
maxlen =   28#@param {type: "number"}
step =   1#@param {type: "number"}
批大小 = 2048 #@param {type: "number"}
总迭代数 = 1024 #@param {type: "number"}
学习率 = 0.001 #@param {type: "number"}
编码 = "GB2312" #@param ["GB2312", "pq"]
是否使用训练集作为编码 = False #@param {type: "boolean"}
训练集 = "yja"  #@param ["nl", "ypzs", "dqol", "tzsy", "zj", "yja"]
是否继续上一次的训练 = False  #@param {type: "boolean"}
模型路径 = "/content/drive/My Drive/AIWritter/Models/yja-004-5.9229" #@param {type: "string"}
训练时最大书写长度 = 100 #@param {type: "integer"}
#@markdown ---

path = f'/content/drive/My Drive/AIWritter/Books/{训练集}.txt'
checkpoint_path = f'/content/drive/My Drive/AIWritter/Models/{训练集}'+'-{epoch:03d}-{loss:.4f}'
with open(path, encoding='UTF-8') as f:
    text = f.read()
print('text length:', len(text))

path = f'/content/drive/My Drive/AIWritter/{编码}.txt' if not 是否使用训练集作为编码 else path
with open(path, encoding='UTF-8') as f:
    charset = f.read()
chars = sorted(list(set(charset)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

for char in text:
    if not char in char_indices.keys():
        char_indices[char] = len(chars)
        indices_char[len(chars)] = char
        chars.append(char)
        with open(path, 'a') as f:
            f.write(char)

# cut the text in semi-redundant sequences of maxlen characters
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: LSTM
try:
    if (not 是否继续上一次的训练):
        raise Exception('不需要载入模型')
    model = load_model(模型路径)
except Exception as e:
    print(repr(e))
    print('Build model...')
    model = Sequential(name='EZWritter')
    model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
    model.add(LSTM(1024, return_sequences=True))
    model.add(LSTM(2048))
    model.summary()
    model.add(Dense(len(chars), activation='softmax'))

    optimizer = RMSprop(learning_rate=学习率)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# show model structure
model.summary()


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    diversity = np.random.randint(1, 3) / 10.0
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(训练时最大书写长度):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(x, y,
          batch_size=批大小,
          epochs=总迭代数,
          callbacks=[model_checkpoint_callback, print_callback])
